<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/SN_web_lightmode.png" width="300">
</center>


<h1>Analysis of Global COVID-19 Pandemic Data</h1>

Estimated time needed: **90** minutes



## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [ ]:
# This lab requires 'httr' and 'rvest'packages, which are already pre-loaded into this lab environment.
# However, if you are working on your local RStudio, please uncomment the below codes and install the packages.

#install.packages("httr")
#install.packages("rvest")

In [ ]:
library(httr)
library(rvest)

Note: if you can import above libraries, please use install.packages() to install them first.


## TASK 1: Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this 

URL https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [ ]:
get_wiki_covid19_page <- function() {
  # Target page is the template transcluded into country pages
  wiki_base_url <- "https://en.wikipedia.org/w/index.php"
  query <- list(title = "Template:COVID-19_testing_by_country")
  # Politely identify as a student script
  resp <- httr::GET(url = wiki_base_url, query = query, httr::user_agent("covid19-lab/1.0"))
  httr::stop_for_status(resp)
  return(resp)
}


Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [ ]:
# Call the get_wiki_covid19_page function and print the response
wiki_response <- get_wiki_covid19_page()
print(wiki_response)


## TASK 2: Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [ ]:
# Get the root html node from the http response in task 1 
root_node <- wiki_response %>% content(as = "text") %>% read_html()
root_node


Get the tables in the HTML root node using `html_nodes` function.


In [ ]:
# Get the table node from the root html node
table_node <- html_node(root_node, "table.wikitable")
table_node


Read the specific table from the multiple tables in the `table_node` using the `html_table` function and convert it into dataframe using `as.data.frame`

_Hint:- Please read the `table_node` with index 2(ex:- table_node[2])._


In [ ]:
# Read the table node and convert it into a data frame, and print the data frame for review
covid_data_frame <- html_table(table_node, fill = TRUE)
# Quick peek
head(covid_data_frame)


## TASK 3: Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame


In [ ]:
# Print the summary of the data frame
summary(covid_data_frame)
str(covid_data_frame)


As you can see from the summary, the columns names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`. 

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [ ]:
preprocess_covid_data_frame <- function(data_frame) {
  # Ensure base types
  data_frame <- as.data.frame(data_frame, stringsAsFactors = FALSE)
  shape <- dim(data_frame)
  # Remove the World row
  if ("Country.or.region" %in% names(data_frame)) {
    data_frame <- data_frame[!(data_frame$`Country.or.region` == "World"), ]
  }
  # Remove the last row if it is an aggregate/notes row and keep only the first 172 as per lab hint
  if (nrow(data_frame) >= 172) {
    data_frame <- data_frame[1:172, ]
  }
  # Drop columns we do not need if present
  drop_cols <- intersect(c("Ref.", "Ref", "Units.b.", "Notes"), names(data_frame))
  if (length(drop_cols)) data_frame[drop_cols] <- NULL
  # Standardize names for easier handling
  names(data_frame) <- make.names(names(data_frame))
  # Attempt to map typical column names from the wiki
  rename_map <- c(
    "Country.or.region" = "country",
    "Country" = "country",
    "Location" = "country",
    "As.of" = "date",
    "Date" = "date",
    "Tested" = "tested",
    "Tests" = "tested",
    "Total.tests" = "tested",
    "Tests.performed" = "tested",
    "Confirmed.cases" = "confirmed",
    "Cases" = "confirmed",
    "Confirmed" = "confirmed",
    "Population" = "population",
    "Confirmed.cases.as.a.proportion.of.tests" = "confirmed.tested.ratio",
    "Tests.as.a.proportion.of.population" = "tested.population.ratio",
    "Confirmed.cases.as.a.proportion.of.population" = "confirmed.population.ratio"
  )
  for (old in names(rename_map)) {
    if (old %in% names(data_frame)) {
      names(data_frame)[names(data_frame) == old] <- rename_map[[old]]
    }
  }
  # Helper to clean numeric strings
  clean_num <- function(x) {
    x <- gsub(",", "", x)
    x <- gsub("\[[^\]]*\]", "", x)   # remove footnotes like [a]
    x <- gsub("[^0-9.]", "", x)
    suppressWarnings(as.numeric(x))
  }
  # Convert column data types
  if ("country" %in% names(data_frame)) data_frame$country <- as.factor(trimws(data_frame$country))
  if ("date" %in% names(data_frame)) data_frame$date <- as.factor(trimws(as.character(data_frame$date)))
  if ("tested" %in% names(data_frame)) data_frame$tested <- clean_num(data_frame$tested)
  if ("confirmed" %in% names(data_frame)) data_frame$confirmed <- clean_num(data_frame$confirmed)
  if ("population" %in% names(data_frame)) data_frame$population <- clean_num(data_frame$population)
  # Convert precomputed ratios if present
  if ("confirmed.tested.ratio" %in% names(data_frame)) data_frame$confirmed.tested.ratio <- clean_num(data_frame$confirmed.tested.ratio)
  if ("tested.population.ratio" %in% names(data_frame)) data_frame$tested.population.ratio <- clean_num(data_frame$tested.population.ratio)
  if ("confirmed.population.ratio" %in% names(data_frame)) data_frame$confirmed.population.ratio <- clean_num(data_frame$confirmed.population.ratio)
  # Derive ratios if missing
  if (!("confirmed.tested.ratio" %in% names(data_frame)) && all(c("confirmed","tested") %in% names(data_frame))) {
    data_frame$confirmed.tested.ratio <- ifelse(data_frame$tested > 0, data_frame$confirmed / data_frame$tested, NA_real_)
  }
  if (!("confirmed.population.ratio" %in% names(data_frame)) && all(c("confirmed","population") %in% names(data_frame))) {
    data_frame$confirmed.population.ratio <- ifelse(data_frame$population > 0, data_frame$confirmed / data_frame$population, NA_real_)
  }
  if (!("tested.population.ratio" %in% names(data_frame)) && all(c("tested","population") %in% names(data_frame))) {
    data_frame$tested.population.ratio <- ifelse(data_frame$population > 0, data_frame$tested / data_frame$population, NA_real_)
  }
  return(data_frame)
}


Call the `preprocess_covid_data_frame` function


In [ ]:
# call `preprocess_covid_data_frame` function and assign it to a new data frame
covid_data_frame <- preprocess_covid_data_frame(covid_data_frame)
head(covid_data_frame)


Get the summary of the processed data frame again


In [ ]:
# Print the summary of the processed data frame again
summary(covid_data_frame)
str(covid_data_frame)


After pre-processing, you can see the columns and columns names are simplified, and columns types are converted into correct types.


The data frame has following columns:

- **country** - The name of the country
- **date** - Reported date
- **tested** - Total tested cases by the reported date
- **confirmed** - Total confirmed cases by the reported date
- **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
- **tested.population.ratio** - The ratio of tested cases to the population of the country
- **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file. 


In [ ]:
# Export the data frame to a csv file
write.csv(covid_data_frame, file = "covid.csv", row.names = FALSE)


Note for IBM Waston Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [ ]:
# Get working directory
wd <- getwd()
# Get exported 
file_path <- paste(wd, sep="", "/covid.csv")
# File path
print(file_path)
file.exists(file_path)

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [ ]:
## Download a sample csv file (uncomment to download if running online)
# covid_csv_file <- download.file(
#   "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/coursera/DA0101EN/coursera/project/FinalProject-RP0101EN-Coursera/v2/dataset/covid.csv",
#   destfile = "covid.csv", quiet = TRUE
# )
# Read the saved CSV
covid_data_frame_csv <- read.csv("covid.csv", header = TRUE, sep = ",", stringsAsFactors = FALSE)
head(covid_data_frame_csv)


## TASK 4: Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [ ]:
# Read covid_data_frame_csv from the csv file
covid_data_frame_csv <- read.csv("covid.csv", header = TRUE, sep = ",", stringsAsFactors = FALSE)

# Get the 5th to 10th rows, with two "country" "confirmed" columns
subset_5_10 <- covid_data_frame_csv[5:10, c("country", "confirmed")]
subset_5_10


## TASK 5: Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [ ]:
# Get the total confirmed cases worldwide
total_confirmed <- sum(covid_data_frame_csv$confirmed, na.rm = TRUE)

# Get the total tested cases worldwide
total_tested <- sum(covid_data_frame_csv$tested, na.rm = TRUE)

# Get the positive ratio (confirmed / tested)
world_positive_ratio <- ifelse(total_tested > 0, total_confirmed / total_tested, NA_real_)

total_confirmed
total_tested
world_positive_ratio


## TASK 6: Get a country list which reported their testing data 

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [ ]:
# Get the `country` column
countries <- covid_data_frame_csv$country

# Check its class (should be Factor or character)
class(countries)

# Sort the countries AtoZ
countries_sorted_AtoZ <- sort(unique(as.character(countries)), decreasing = FALSE)

# Sort the countries ZtoA
countries_sorted_ZtoA <- sort(unique(as.character(countries)), decreasing = TRUE)

# Print the sorted ZtoA list
countries_sorted_ZtoA


## TASK 7: Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires start with `United`


In [ ]:
# Use a regular expression `United.+` to find matches
countries_chr <- unique(as.character(covid_data_frame_csv$country))
united_countries <- grep("^United.+", countries_chr, value = TRUE)

# Print the matched country names
united_countries


## TASK 8: Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [ ]:
# Select a subset (should be only one row) of data frame based on a selected country name and columns
country_1 <- "Indonesia"
country_2 <- "Singapore"

country_subset <- subset(covid_data_frame_csv, country %in% c(country_1, country_2),
                         select = c(country, confirmed, confirmed.population.ratio))
country_subset


## TASK 9: Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [ ]:
# Use if-else statement to compare selected countries' confirmed.population.ratio
if (nrow(country_subset) == 2) {
  a <- country_subset$confirmed.population.ratio[country_subset$country == country_1]
  b <- country_subset$confirmed.population.ratio[country_subset$country == country_2]
  if (is.na(a) || is.na(b)) {
    print("One of the ratios is NA; cannot compare.")
  } else if (a > b) {
    print(paste(country_1, "has a higher confirmed-to-population ratio than", country_2))
  } else if (a < b) {
    print(paste(country_2, "has a higher confirmed-to-population ratio than", country_1))
  } else {
    print("Both countries have the same confirmed-to-population ratio.")
  }
} else {
  print("Expected exactly two rows in country_subset.")
}


## TASK 10: Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [ ]:
# Get a subset of any countries with `confirmed.population.ratio` less than the threshold
threshold <- 0.001  # 0.1% of population
low_risk_countries <- subset(covid_data_frame_csv, confirmed.population.ratio < threshold,
                             select = c(country, confirmed, population, confirmed.population.ratio))
# Show the first few results
head(low_risk_countries[order(low_risk_countries$confirmed.population.ratio, decreasing = FALSE), ], 15)
